# Demonstration


In [1]:
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
import pandas as pd
# df = pd.read_csv('data_initial/options_2024_expiry_dates.csv', index_col = [0])
df = pd.read_csv('data_initial/expiry.csv', index_col = [0])
df.reset_index(inplace = True)
df['expiry_dates'] = pd.to_datetime(df['expiry_dates'], format='%d-%m-%Y').dt.strftime('%d-%b-%Y')
# fd = pd.read_csv('data_initial/options_2024_CE_strike_price.csv', index_col = [0])
fd = pd.read_csv('data_initial/strike_price.csv', index_col = [0])

fd = fd[1:].copy()

fd.reset_index(inplace = True)
# fd['Strike prices'] = fd['Strike prices'].str.replace(',', '').astype(float)
# fd = fd[(fd['Strike prices']> 21850) & (fd['Strike prices']< 26500) ].copy() 

fd['Strike prices'] = fd['Strike prices'].apply(lambda x: f"{x:,.2f}")
fd['Strike prices'] = fd['Strike prices'].str.replace(',','')
fd = fd[
    fd['Strike prices'].astype(float) < 25000 ][
    fd[fd['Strike prices'].astype(float) < 25000 
      ]['Strike prices'].astype(float) >22900]

In [7]:
fin = pd.DataFrame([(x, y) for x in df['expiry_dates'] for y in fd['Strike prices']], columns=['expiry_dates', 'Strike prices'])
fin['year'] = '2024'
# fin['typ'] = 'CE'
fin['typ'] = 'PE'
fin = fin [['year','expiry_dates','typ','Strike prices']]
fin = fin.iloc[::-1].reset_index(drop=True).copy()

In [8]:
# fd = pd.read_csv('results.csv')

def range_strategy(df,fd):
    fd['acc'] = 0
    hist = pd.DataFrame()
    for i, row in fd.iterrows():
        acc = 0
        diff_val = 0
        if i == (len(fd) - 1):
            break
        presd = df[df.date == row.date].iloc[0]
        nexd  = df.iloc[presd.name + 1]
        try :
            presd['LAST PRICE'] = presd['LAST PRICE'].replace(',','')
            nexd['LOW PRICE'] = nexd['LOW PRICE'].replace(',','')
            nexd['HIGH PRICE'] = nexd['HIGH PRICE'].replace(',', '')
        except:
            continue
        
        if float(row.sen_day_break) == 0:
            if float(presd['LAST PRICE']) > float(nexd['LOW PRICE']):
                fd.loc[fd.date == row.date, 'acc'] = 1
                acc = 1
                diff_val = float(presd['LAST PRICE']) - float(nexd['LOW PRICE'])
            else:
                diff_val = float(presd['LAST PRICE']) - float(nexd['LOW PRICE'])
        else:
            if float(presd['LAST PRICE']) < float(nexd['HIGH PRICE']):
                fd.loc[fd.date == row.date, 'acc'] = 1
                acc = 1
                diff_val = float(nexd['HIGH PRICE']) - float(presd['LAST PRICE'])
            else:
                diff_val = float(nexd['HIGH PRICE'])- float(presd['LAST PRICE'])
        hist = pd.concat([hist, pd.DataFrame([[
            float(presd['LAST PRICE']),
            float(nexd['HIGH PRICE']),
            float(nexd['LOW PRICE']),
            diff_val,
            row.sen_day_break,
            presd.date,
            nexd.date,
            acc
            ]],
            columns = ['plast', 'nhigh', 'nlow','diff', 'sentiment','present_date', 'predictive_date','acc']
        )] , ignore_index = True)
    print(sum(fd.acc)/ (len(fd)-1), len(fd))
    return hist

def absolute_algo(df,fd):
    fd['acc'] = 0
    for i, row in fd.iterrows():
        
        if i == len(fd) - 1 :
            break
        presd = df[df.date == row.date].iloc[0]
        nexd  = df.iloc[presd.name + 1]
        if row.sen_day_break == 0:
            if presd['CLOSE PRICE'] > nexd['CLOSE PRICE']:
                fd.loc[fd.date == row.date, 'acc'] = 1
        else :
            if presd['CLOSE PRICE'] < nexd['CLOSE PRICE']:
                fd.loc[fd.date == row.date, 'acc'] = 1
    print(sum(fd.acc)/ len(fd), len(fd))

    return 
hist_arr = []
# for i, row in pd.DataFrame('data_initial/call/' + fin['year'] + '_'+fin['expiry_dates'] + '_'+ fin['typ'] +'_'+ fin['Strike prices']+'.csv').iterrows():
for i, row in pd.DataFrame('data_initial/put/' + fin['year'] + '_'+fin['expiry_dates'] + '_'+ fin['typ'] +'_'+ fin['Strike prices']+'.csv').iterrows():
    df = pd.read_csv(str(row[0]))
    df=df.iloc[::-1].reset_index(drop=True)
    df = df[(df != '-').all(axis=1)]
    df['date'] = pd.to_datetime(df['DATE'], format='%d-%b-%Y')
    
    fd = pd.read_csv('results_NSEI.csv')
    fd['date'] = pd.to_datetime(fd['date']).dt.strftime('%Y-%m-%d')
    fd['acc'] = 0
    fd['date'] = pd.to_datetime(fd['date'])
    fd = fd[fd['date'].isin(df['date'])]
    df.reset_index(inplace = True, drop = True)
    fd.reset_index(inplace = True, drop = True)
    fdd = fd.copy()
    hist = range_strategy(df,fd)
    hist.to_csv(str(row[0]).replace('call/','call/results/').replace('put/','put/results/'))
    hist_arr.append(hist)

    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


0.75 9
0.7692307692307693 14
0.7272727272727273 12
0.8461538461538461 14
0.6666666666666666 13
0.9285714285714286 15
0.6923076923076923 14
0.8571428571428571 15
0.8461538461538461 14
0.8571428571428571 15
0.8333333333333334 13
0.8 16
0.8333333333333334 13
0.8666666666666667 16
0.8461538461538461 14
0.875 17
0.8571428571428571 15
0.8571428571428571 15
0.8333333333333334 13
0.8125 17
0.8333333333333334 13
0.8571428571428571 15
0.8333333333333334 13
0.9285714285714286 15
0.8333333333333334 13
0.0 17
1.0 9
0.8571428571428571 15
0.0 13
0.8 16
0.0 16
0.8181818181818182 12
0.0 15
0.0 17
0.0 11
0.8333333333333334 13
0.875 9
0.7692307692307693 14
0.875 9
0.8571428571428571 15
1.0 9


In [5]:
diff_mean_arr_call = []
diff_median_arr_call = []
accu_arr_call = []
for i in hist_arr:
    if len(i) == 0:
        continue
    diff_median_arr_call.append(i['diff'].median())
    diff_mean_arr_call.append(i['diff'].mean())
    accu_arr_call.append(sum(i['acc'])/len(i['acc']))

In [9]:
diff_mean_arr_put = []
diff_median_arr_put = []
accu_arr_put = []
for i in hist_arr:
    if len(i) == 0:
        continue
    diff_median_arr_put.append(i['diff'].median())
    diff_mean_arr_put.append(i['diff'].mean())
    accu_arr_put.append(sum(i['acc'])/len(i['acc']))

In [10]:
summary = pd.DataFrame(
    [[
    sum(diff_median_arr_call)/len(diff_median_arr_call),
    sum(diff_mean_arr_call)/len(diff_mean_arr_call),
    sum(accu_arr_call)/len(accu_arr_call),
    sum(diff_median_arr_put)/len(diff_median_arr_put),
    sum(diff_mean_arr_put)/len(diff_mean_arr_put),
    sum(accu_arr_put)/len(accu_arr_put)
    ]], columns = [
        'avg_diff_median_call',
        'avg_diff_mean_call',
        'avg_acc_call',
        'avg_diff_median_put',
        'avg_diff_mean_put',
        'avg_acc_put'
    ]
)

In [11]:
summary

,avg_diff_median_call,avg_diff_mean_call,avg_acc_call,avg_diff_median_put,avg_diff_mean_put,avg_acc_put
0,50.362821,60.77275,0.894351,55.389286,64.100704,0.837886


In [12]:
summary.to_csv('data_initial/summary/summary.csv')